In [1]:

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
from scipy.signal import find_peaks, iirnotch
import os
import pandas as pd
from vmdpy import VMD
from scipy import signal as sig
from scipy.stats import pearsonr
import sys
from scipy.signal import butter, filtfilt
from concurrent.futures import ThreadPoolExecutor
from scipy.signal import lombscargle

In [2]:
ecg_fps = 2000
ppg_fps = 50
rPPG_fps = 30

ecg_peak_height = 0.1

convolve_mode = 'same'

ROOT_PATH = r"F:\hrv_dataset"


In [3]:
# ECG信号处理

def bandpass_filter(signal, order=4):
# 带通滤波器，用于消除肌电干扰。
    lowcut = 0.5 
    highcut = 50
    fs = 2000
    nyquist = 0.5 * fs # 奈奎斯特频率
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

# 陷波滤波器设计
def notch_filter(signal, quality=30):
    freq = 50
    fs = ecg_fps
    # 陷波滤波器，用于消除工频干扰（50 Hz）。
    nyquist = 0.5 * fs
    freq = freq / nyquist
    b, a = iirnotch(freq, quality)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

# 滤-基线漂移
def baseline_filter(data, fs=ecg_fps):
    data = np.array(data)
    winsize = int(round(0.2*fs))
    if winsize % 2 == 0:
        winsize += 1
    baseline_estimate = sig.medfilt(data, kernel_size=winsize)
    winsize = int(round(0.6*fs))
    if winsize % 2 == 0:
        winsize += 1
    baseline_estimate = sig.medfilt(baseline_estimate, kernel_size=winsize)
    ecg_blr = data - baseline_estimate
    return ecg_blr.tolist()

def ecg_smooth(signal, window_len=100, window='hanning'):
    if window_len > len(signal):
        return signal
    window = getattr(np, window)(window_len)
    return np.convolve(signal, window, mode=convolve_mode)[:len(signal)]

In [4]:
# PPG信号处理
def ppg_smooth(signal, window_len=25, window='hanning'):
    if window_len > len(signal):
        return signal
    window = getattr(np, window)(window_len)
    smoothed_signal = np.convolve(signal, window, mode=convolve_mode)[:len(signal)]
    return smoothed_signal

In [10]:
# rPPG信号处理
def rPPG_smooth(signal, window_len=5, window='hanning'):
    if window_len > len(signal):
        return signal
    window = getattr(np, window)(window_len)
    smoothed_signal = np.convolve(signal, window, mode=convolve_mode)[:len(signal)]
    return smoothed_signal

In [6]:
# 生成虚拟信号
def generate_vSignal(smoothed_signal, fps, peak_height=None):
    peaks, _ = find_peaks(smoothed_signal, height=peak_height, distance=fps*0.4)
    peak_indices = np.array(peaks)
    vSignal = np.zeros_like(smoothed_signal)
    num_peaks = len(peak_indices)
    # 处理第一个周期，确保从信号开始就有连续的余弦波形
    if num_peaks > 1:
        start = peak_indices[0]
        end = peak_indices[1]
        duration = end - start
        x = np.linspace(0, duration, num=end + 1)  # end+1 确保包含 end 索引
        vSignal[:end + 1] = np.cos(2 * np.pi * (x - 0) / duration)
    # 处理中间的周期
    for i in range(1, num_peaks - 1):
        start = peak_indices[i]
        end = peak_indices[i + 1]
        duration = end - start
        x = np.linspace(start, end, num=end - start + 1)
        vSignal[start:end + 1] = np.cos(2 * np.pi * (x - start) / duration)
    # 处理最后一个周期，确保余弦波形在信号末尾也是连续的
    if num_peaks > 1:
        start = peak_indices[-1]
        duration = len(smoothed_signal) - start
        x = np.linspace(0, duration, num=len(smoothed_signal) - start)
        vSignal[start:] = np.cos(2 * np.pi * (x - 0) / duration)
    return vSignal

def butter_bandpass_filter(signal, lowcut=0.95, highcut=2.2, fs=None, order=5):
    """Butterworth 带通滤波"""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# 归一化
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

In [7]:
# 计算RR间期
def cal_rr_intervals(signal, fps):
    if fps == ecg_fps:
        peaks, _ = find_peaks(signal)
    else:
        peaks, _ = find_peaks(signal)
    rr_intervals = np.diff(peaks) / fps
    return rr_intervals


# 计算HRV的时域指标
def cal_time_domain_hrv(rr_intervals):
    mean_rr = np.mean(rr_intervals)
    sdnn = np.std(rr_intervals, ddof=0)
    rmssd = np.sqrt(np.mean(np.diff(rr_intervals) ** 2))
    n50 = np.sum(np.abs(np.diff(rr_intervals)) > 0.05)  # 50ms转换为秒
    pnn50 = (n50 / (len(rr_intervals) - 1)) * 100
    return {
        'Mean RR': mean_rr,
        'SDNN': sdnn,
        'RMSSD': rmssd,
        'NN50': n50,
        'pNN50': pnn50
    }


# 计算HRV的频域指标
def cal_frequency_domain_hrv(rr_intervals, target_fps=4):
    rr_timestamps = np.cumsum(rr_intervals)
    rr_timestamps -= rr_timestamps[0]
    frequencies = np.linspace(0.01, 0.5, 1000)  # 0.01 Hz到0.5 Hz
    power = lombscargle(rr_timestamps, rr_intervals, frequencies, normalize=True)
    lf_range = (frequencies >= 0.04) & (frequencies <= 0.15)  # 低频范围 (0.04-0.15 Hz)
    hf_range = (frequencies >= 0.15) & (frequencies <= 0.4)   # 高频范围 (0.15-0.4 Hz)
    plf = np.trapz(power[lf_range], frequencies[lf_range])  # 低频功率
    phf = np.trapz(power[hf_range], frequencies[hf_range])  # 高频功率
    lf_hf_ratio = plf / phf  # LF/HF比值
    return {
        'LF': plf,
        'HF': phf,
        'LF/HF': lf_hf_ratio
    }

In [8]:
def cal_hrv(folder_name, vDATA, fps, window_size=300, step=5):
    # 计算时域，频域
    time_domains = [] 
    frequency_domains = []
    window_size *= fps
    step *= fps
    for window in range(0, len(vDATA) - window_size, step):
        rr_intervals = cal_rr_intervals(vDATA[window:window+window_size], fps)
        hrv_time_domain = cal_time_domain_hrv(rr_intervals)
        hrv_frequency_domain = cal_frequency_domain_hrv(rr_intervals)
        time_domains.append(hrv_time_domain)
        frequency_domains.append(hrv_frequency_domain)
    
    # 合并时域，频域
    merged_data = []
    for time_domain, frequency_domain in zip(time_domains, frequency_domains):
        merged_data.append({**time_domain, **frequency_domain})

    dict = {
        'Name': f'{folder_name}',
        'Mean RR': [d['Mean RR'] for d in merged_data],
        'SDNN': [d['SDNN'] for d in merged_data],
        'RMSSD': [d['RMSSD'] for d in merged_data],
        'NN50': [d['NN50'] for d in merged_data],
        'pNN50': [d['pNN50'] for d in merged_data],
        'LF': [d['LF'] for d in merged_data],
        'HF': [d['HF'] for d in merged_data],
        'LF/HF': [d['LF/HF'] for d in merged_data]
    }

    return dict

In [9]:
ROOT_PATH = 'F:\\hrv_dataset'
ecg_hrv_path = os.path.join(ROOT_PATH, 'ecg_hrv.csv')
ppg_hrv_path = os.path.join(ROOT_PATH, 'ppg_hrv.csv')
rPPG_hrv_path = os.path.join(ROOT_PATH, 'rPPG_hrv.csv')


def process_folder(folder):
    """多线程处理单个文件夹的核心函数"""
    # print(folder)
    folder_path = os.path.join(ROOT_PATH, folder)
    ecg_dict, ppg_dict, rPPG_dict = None, None, None

    # ECG处理
    ecg_file = os.path.join(folder_path, 'ECG_EEG_RSP.csv')
    if os.path.exists(ecg_file):
        try:
            ECG_df = pd.read_csv(ecg_file, header=0)[['ECG, X, RSPEC-R']]
            ECGs = ECG_df.values.flatten()
            filter_01_data = bandpass_filter(ECGs)
            filter_02_data = notch_filter(filter_01_data)
            filter_03_data = baseline_filter(filter_02_data)
            ecg_smoothed = ecg_smooth(filter_03_data)
            vECG = generate_vSignal(ecg_smoothed, ecg_fps, ecg_peak_height).flatten()
            ecg_dict = cal_hrv(folder, vECG, ecg_fps)
        except Exception as e:
            print(f"ECG Error in {folder}: {str(e)}")

    # PPG处理
    ppg_file = os.path.join(folder_path, 'ppg.csv')
    if os.path.exists(ppg_file):
        try:
            PPG_df = pd.read_csv(ppg_file, header=0, usecols=['ppg'])
            PPGs = PPG_df.values.flatten()
            ppg_smoothed = ppg_smooth(PPGs)
            vPPG = generate_vSignal(ppg_smoothed, ppg_fps).flatten()
            ppg_dict = cal_hrv(folder, vPPG, ppg_fps)
        except Exception as e:
            print(f"PPG Error in {folder}: {str(e)}")

    # rPPG处理
    rPPG_file = os.path.join(folder_path, 'rPPG_bigger.csv')
    if os.path.exists(rPPG_file):
        try:
            rPPG_df = pd.read_csv(rPPG_file)
            G_col = rPPG_df['G'].values.flatten()
            R_col = rPPG_df['R'].values.flatten()
            R_col = np.clip(R_col, 1e-6, None)  # 避免除以0
            B_col = rPPG_df['B'].values.flatten()
            B_col = np.clip(B_col, 1e-6, None)
            rPPGs = G_col/R_col + G_col/B_col
            rPPG_smoothed = rPPG_smooth(rPPGs)
            rPPG_smoothed = butter_bandpass_filter(signal=rPPG_smoothed, fs=rPPG_fps)
            vRPPG = generate_vSignal(rPPG_smoothed, rPPG_fps).flatten()
            rPPG_dict = cal_hrv(folder, vRPPG, rPPG_fps)
        except Exception as e:
            print(f"rPPG Error in {folder}: {str(e)}")

    return ecg_dict, ppg_dict, rPPG_dict

folders = [f for f in os.listdir(ROOT_PATH) if os.path.isdir(os.path.join(ROOT_PATH, f))]

# 线程池
with ThreadPoolExecutor(max_workers=16) as executor:
    results = executor.map(process_folder, folders)

# 收集结果
ecg_dicts, ppg_dicts, rPPG_dicts = [], [], []
for ecg_dict, ppg_dict, rPPG_dict in results:
    if ecg_dict: ecg_dicts.append(ecg_dict)
    if ppg_dict: ppg_dicts.append(ppg_dict)
    if rPPG_dict: rPPG_dicts.append(rPPG_dict)

# 保存结果
pd.DataFrame(ecg_dicts).to_csv(ecg_hrv_path, index=False)
pd.DataFrame(ppg_dicts).to_csv(ppg_hrv_path, index=False)
pd.DataFrame(rPPG_dicts).to_csv(rPPG_hrv_path, index=False)


rPPG Error in s75: The length of the input vector x must be greater than padlen, which is 33.
